In [37]:
from __future__ import print_function, division
import pandas as pd
from meye import MEImage
from scipy import signal as sg
from scipy.ndimage.filters import maximum_filter
%pylab inline
plt.rcParams['image.cmap'] = 'gray'

Populating the interactive namespace from numpy and matplotlib


In [79]:
df = pd.read_pickle("store.pickle")
df.head()

,Z,currImage,currRect,egoMotion,focal,prevImage,prevRect
0,13.85,16-06-05_17-25-17_Single_0026_frame52.img,"[117, 202, 650, 736]","[[0.999999578399109, -0.000895476002401953, -0...",1612,16-06-05_17-25-17_Single_0026_frame51.img,"[200, 268, 625, 701]"
1,16.7,16-06-05_17-25-17_Single_0026_frame51.img,"[200, 268, 625, 701]","[[0.9999966311214874, -0.0025956984235227477, ...",1612,16-06-05_17-25-17_Single_0026_frame50.img,"[257, 317, 615, 677]"
2,None,16-06-05_17-25-17_Single_0026_frame50.img,"[257, 317, 615, 677]","[[0.9999998869381259, 0.00046968847719411973, ...",1612,16-06-05_17-25-17_Single_0026_frame49.img,"[299, 351, 605, 659]"


# step 1

In [81]:
sobel_kernel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]) / 8
sobel_kernel_y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]]) / 8

In [83]:
line_zero = df.iloc[0]
curr_rect = one_ex.currRect
prev_rect = one_ex.prevRect

In [84]:
cimage = MEImage.from_file(one_ex.currImage)
pimage = MEImage.from_file(one_ex.prevImage)

In [86]:
# plt.imshow(pimage.im,cmap='gray',origin='lower')
# plt.imshow(cimage.im,cmap='gray')

In [88]:
prev_rect_mat = pimage.im[prev_rect[2]:prev_rect[3],prev_rect[0]:prev_rect[1]]
curr_rect_mat = cimage.im[curr_rect[2]:curr_rect[3],curr_rect[0]:curr_rect[1]]

In [89]:
# plt.imshow(prev_rect_mat,cmap='gray',origin='lower')
# plt.imshow(curr_rect_mat,cmap='gray',origin='lower')

In [90]:
prev_dx = sg.convolve2d(prev_rect_mat, sobel_kernel_x, "same")
prev_dy = sg.convolve2d(prev_rect_mat, sobel_kernel_y, "same") 

In [91]:
# plt.imshow(prev_dx,cmap='gray',origin='lower')
# plt.imshow(prev_dy,cmap='gray',origin='lower')

In [94]:
max_prev_dx = maximum_filter(prev_dx, 5)
max_prev_dy = maximum_filter(prev_dy, 5)

In [96]:
# plt.imshow(max_prev_dx,cmap='gray',origin='lower')
# plt.imshow(max_prev_dy,cmap='gray',origin='lower')

# step 2